In [62]:
import requests  ## for getting data from a server GET
import re   ## for regular expressions   
import numpy as np
import pandas as pd ## for dataframes and related
from pandas import DataFrame
import graphviz
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
from lxml import html  
import csv
from time import sleep
import argparse
import sys
import time as t
from pathlib import Path

from textblob import TextBlob
import tweepy
import nltk
#import pycountry
import string
from PIL import Image

import datetime
import dateutil.parser
import unicodedata
#To add wait time between requests
import time
import configparser
import tweepy


In [80]:
corpus_path = Path('E:\Masters_Coursework\Sem4\Text Mining\Project\CORPUS')
project_path = Path('E:\Masters_Coursework\Sem4\Text Mining\Project')
labels = ["+Nike +footwear +market +ranking",
"+global +footwear +market +leader",
"+top +brand in +footwear +industry",
"+Nike +market +share in +footwear",
"Nike footwear",
"footwear industry"]
endpoint="https://newsapi.org/v2/everything"
filename="newsApiData.csv"
nytimes_data =  "fav_sneaker_data.txt"
nike_vs_adidas_data =  "nike_vs_adidas_data.txt"
nike_reviews_data = "nike_reviews_data.csv"
twitter_data = "twitter_data.csv"
twitter_labels = ["footwear market",
"global footwear leader",
"top brand in footwear",
"Nike market",
"Nike footwear",
"footwear industry",
"Leading footwear brand"]


NEWS API


In [ ]:
jsontxt = {}
for topic in labels:
    ## Dictionary Structure
    URLPost = {'apiKey':'b1c5c9b6caec41e4933b5c6c76c4051b',
                'sort_by':'relevancy',
               'q':topic,
               'pageSize':100
    }

    response=requests.get(endpoint, URLPost)
    print(response)
    jsontxt[topic] = response.json()
    #print(jsontxt)

In [ ]:
def clean_content(text):
    text=re.sub(r'[,.;@#?!&$\-\']+', ' ', text, flags=re.IGNORECASE)
    text=re.sub(' +', ' ', text, flags=re.IGNORECASE)
    text=re.sub(r'\"', ' ', text, flags=re.IGNORECASE)
    text=re.sub(r'[^a-zA-Z]', " ", text, flags=re.VERBOSE)
    text=text.replace(',', '')
    text=' '.join(text.split())
    text=re.sub("\n|\r", "", text)
    text= ' '.join([wd for wd in text.split() if len(wd)>3])
    return text

file_path = project_path.joinpath(filename)
MyFILE=open(file_path,"w")
WriteThis="Label,Date,Author,Source,Title,Url,Headline,Content\n"
MyFILE.write(WriteThis)
for topic in labels:
    articles = jsontxt[topic]["articles"]
    label = re.sub('[+]',"",topic) if "+" in topic else topic
    for i in range(len(articles)):              
        Date=articles[i]["publishedAt"]
        NewDate=Date.split("T")
        Date=NewDate[0]

        Author=items["author"]
        Author=str(Author)
        Author=Author.replace(',', '')

        Source=articles[i]["source"]["name"]

        Title=articles[i]["title"]
        Title=clean_content(str(Title))
        
        url = articles[i]["url"]

        Headline=articles[i]["description"]
        Headline=clean_content(str(Headline))

        Content=articles[i]["content"]
        Content=clean_content(str(Content))
        
        WriteThis=str(label)+","+str(Date)+","+str(Author)+","+str(Source)+","+ str(Title) + "," + str(Url) + "," + str(Headline) +"," + str(Content) + "\n" 
        MyFILE.write(WriteThis)
            
MyFILE.close()

WEB SCRAPING

In [ ]:
# Websites used for scrapping
# https://www.nytimes.com/wirecutter/guides/our-favorite-sneakers/
# https://www.trustpilot.com/review/www.nike.com 
# https://seekingalpha.com/article/4516865-nike-vs-adidas-an-undisputed-leader
# https://www.feedough.com/10-biggest-nike-competitors/
# https://travel.earth/adidas-vs-nike/ 
# https://www.unacast.com/post/nike-adidas-puma-foot-traffic


In [1]:
def get_website_soupdata(url_link):
    headers = {'User-Agent':'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/88.0.4324.96 Safari/537.36'}
    response = requests.get(url_link,headers=headers).text
    return BeautifulSoup(response,'lxml')

In [9]:
#url = 'https://www.nytimes.com/wirecutter/guides/our-favorite-sneakers/'
nytimes_soup = get_website_soupdata(url)
info = nytimes_soup.find_all('div',attrs = {'class':'ca523f26 _7cb53a9f _614adc05'})
fav_sneakers = []
if info is not None and len(info)>0 :
    for record in info:
        para = record.find('p')
        if para is not None:
            fav_sneakers.append(para.text) 

    if len(fav_sneakers) > 0:
        file_path = corpus_path.joinpath(nytimes_data)
        MyFILE=open(file_path,"w")
        WriteThis="Blog Content\n"
        MyFILE.write(WriteThis)
        MyFILE.writelines(fav_sneakers)
        MyFILE.close() 

In [21]:
url = 'https://travel.earth/adidas-vs-nike/'
adidas_nike_soup = get_website_soupdata(url)
nike_adidas_info = adidas_nike_soup.find_all('div',attrs = {'class':'td-post-content tagdiv-type'})
nike_adidas = []
if nike_adidas_info is not None and len(nike_adidas_info)>0 :
    for record in nike_adidas_info:
        all_para = record.find_all('p')
        for val in all_para:
            if val is not None:
                nike_adidas.append(val.text) 

    if len(nike_adidas) > 0:
        file_path = corpus_path.joinpath(nike_vs_adidas_data)
        MyFILE=open(file_path,"w",encoding="utf-8")
        WriteThis="Blog Content\n"
        MyFILE.write(WriteThis)
        MyFILE.writelines(nike_adidas)
        MyFILE.close() 

In [120]:
url = 'https://www.trustpilot.com/review/www.nike.com'

def get_all_reviews_url(url):
    all_url_list = []
    all_url_list.append(url)
    for i in range(0,15):
        all_url_list.append(url+'?page='+str(i+2))
    return all_url_list

all_review_urls = get_all_reviews_url(url)
file_path = project_path.joinpath(nike_reviews_data)
MyFILE=open(file_path,"w",encoding="utf-8")
WriteThis="Title,Rating,Review\n"
MyFILE.write(WriteThis)
for url in all_review_urls:
    nike_soup = get_website_soupdata(url)
    nike_reviews = nike_soup.find_all('section',attrs = {'class':'styles_reviewContentwrapper__zH_9M'})
    if nike_reviews is not None and len(nike_reviews)>0 :
        
        for record in nike_reviews:
            title_h2 = record.find('h2',attrs = {'class':'typography_heading-s__f7029 typography_appearance-default__AAY17'})
            rating_img = record.find('img')
            para_p = record.find('p',attrs = {'class':'typography_body-l__KUYFJ typography_appearance-default__AAY17 typography_color-black__5LYEn'})
            WriteThis= (str(title_h2.text) if title_h2 is not None else 'N/A') + "," + (str(rating_img['alt']) if rating_img is not None else 'N/A') + "," + (str(para_p.text) if para_p is not None else 'N/A')+"\n"   
            MyFILE.write(WriteThis)       
        ## CLOSE THE FILE
MyFILE.close()

TWITTER API

In [63]:
# read config
config = configparser.ConfigParser()
config.read('config.ini')

api_key = config['twitter']['consumerKey']
api_key_secret = config['twitter']['consumerSecret']

access_token = config['twitter']['accessToken']
access_token_secret = config['twitter']['accessTokenSecret']


# authenticate
auth = tweepy.OAuthHandler(api_key, api_key_secret)
auth.set_access_token(access_token, access_token_secret)
api = tweepy.API(auth)

In [77]:
tweets_jsontxt = {}
for topic in twitter_labels:
    topic = re.sub('[+]',"",topic) if "+" in topic else topic
    try:
        fetched_tweets = api.search_tweets(q = topic,lang="en", count = 100,text: "full_text")
        tweets_jsontxt[topic] = fetched_tweets
    except:
        pass

In [119]:
def clean_content(text):
    text=re.sub(r'[,.;@#?!&$\-\']+', ' ', text, flags=re.IGNORECASE)
    text=re.sub(' +', ' ', text, flags=re.IGNORECASE)
    text=re.sub(r'\"', ' ', text, flags=re.IGNORECASE)
    text=re.sub(r'[^a-zA-Z]', " ", text, flags=re.VERBOSE)
    text=text.replace(',', '')
    text=' '.join(text.split())
    text=re.sub("\n|\r", "", text)
    text= ' '.join([wd for wd in text.split() if len(wd)>3])
    return text

file_path = project_path.joinpath(twitter_data)
MyFILE=open(file_path,"w")
WriteThis="Label,Tweet ID,Tweet Text\n"
MyFILE.write(WriteThis)
tweets =[]
for topic in twitter_labels:
    if topic in tweets_jsontxt:
        fetched_tweets = tweets_jsontxt[topic]
        # parsing tweets one by one
        for tweet in fetched_tweets:
            text = clean_content(str(tweet.text))
            tweet_id = tweet.id    
            # if tweet has retweets, ensure that it is appended only once
            if text not in tweets:
                tweets.append(text)
                WriteThis=str(topic)+","+str(tweet_id)+","+str(text)+"\n" 
                MyFILE.write(WriteThis)            
MyFILE.close()



In [102]:
#fetched_tweets = api.search_tweets(q = "poonam",lang="en", count = 100,tweet_mode='extended')